**INTRODUÇÃO**

O primeiro passo deste notebook é a coleta as letras de músicas de Gilberto Gil disponíveis no site [letras.mus.br](https://www.letras.mus.br/gilberto-gil/). E, seguida, são aplicadas técnicas de processamento de linguagem natural para se chegar às palavras mais presentes nas composições e, em seguida, criar uma nuvem de palavras.

**ÍNDICE**

1. [Raspagem de dados](#Raspagem)
2. [Criação de nuvem de palavras](#Nuvem)


**EXPEDIENTE**

**Edição DATADOC**

Thays Lavor

**Coleta de dados**

Alexandre Cajazeira

**Análise e visualização de dados**

Gabriela Custódio

## 1. Raspagem de dados <a name="Raspagem"></a>

In [ ]:
from urllib.error import HTTPError
from urllib.error import URLError
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

In [ ]:
# dados de uma música: 

In [ ]:
song_url = 'https://www.letras.com/gilberto-gil/16137/'
soup = BeautifulSoup(urlopen(song_url), 'html.parser')

In [ ]:
windowPageArgs = soup.find("script",{"id":"head_scripts_vars"}).text.split('__pageArgs=')[1].split(";")[0] 
windowPageArgs

'{"album_name":"Raça Humana","artist_name":"Gilberto Gil","dns":"gilberto-gil","genre":"mpb","lyric_lang":"pt","page":"lyric","track_name":"Vamos Fugir","url":"16137"}'

In [ ]:
stanzas = []
for p in soup.find("div","cnt-letra p402_premium").findAll("p"):
  stanzas.append(
      str(p)
        .replace("<p>","")
        .replace("</p>","")
        .replace("<br>","\n")
        .replace("<br/>","\n")
        .replace("</br>","\n")
    )

lyrics = "\n\n".join(stanzas)

In [ ]:
views = int(soup.find("div","cnt-info_exib").find("b").getText().replace('.',''))
views

7221

In [ ]:
# Função desenvolvida: 

In [ ]:
def getSongData(url):
  soup = BeautifulSoup(urlopen(url), 'html.parser')

  windowPageArgs = soup.find("script",{"id":"head_scripts_vars"}).text.split('__pageArgs=')[1].split(";")[0] 

  views = -1
  views_div = soup.find("div","cnt-info_exib")
  if views_div is not None:
    views = int(views_div.find("b").getText().replace('.',''))

  comp = -1
  comp_div = soup.find("div","letra-info_comp")
  if comp_div is not None:
    comp = comp_div.getText().replace('  Compuesta por: ','').replace('. ¿Los datos están equivocados? Avísanos. ','').replace(' ¿Sabes quién compuso esta canción? Envíanoslo. ', 'Sem informação')

  stanzas = []
  for p in soup.find("div","cnt-letra p402_premium").findAll("p"):
    stanzas.append(
        str(p)
          .replace("<p>","")
          .replace("</p>","")
          .replace("<br>","\n")
          .replace("<br/>","\n")
          .replace("</br>","\n")
      )

  lyrics = "\n\n".join(stanzas)

  return {"windowPageArgs":windowPageArgs,"views":views, "composicao":comp, "lyrics":lyrics}


In [ ]:
getSongData(song_url)

{'composicao': 'Liminha / Gilberto Gil',
 'lyrics': "Vamos fugir!\nDeste lugar\nBaby!\nVamos fugir\nTô cansado de esperar\nQue você me carregue\n\n\nVamos fugir!\nPr'outro lugar\nBaby!\nVamos fugir\nPr'onde quer que você vá\nQue você me carregue\n\nPois diga que irá\nIrajá, Irajá\nPra onde eu só veja você\nVocê veja a mim só\nMarajó, Marajó\nQualquer outro lugar comum\nOutro lugar qualquer\n\nGuaporé, Guaporé\nQualquer outro lugar ao sol\nOutro lugar ao sul\nCéu azul, céu azul\nOnde haja só meu corpo nu\nJunto ao seu corpo nu\n\nVamos fugir!\nPr'outro lugar\nBaby!\nVamos fugir\nPr'onde haja um tobogã\nOnde a gente escorregue\n\nVamos fugir!\nDeste lugar\nBaby!\nAh! Vamos fugir\nTô cansado de esperar\nQue você me carregue\n\nPois diga que irá\nIrajá, Irajá\nPra onde eu só veja você\nVocê veja a mim só\nMarajó, Marajó\nQualquer outro lugar comum\nOutro lugar qualquer\n\nGuaporé, Guaporé\nQualquer outro lugar ao sol\nOutro lugar ao sul\nCéu azul, céu azul\nOnde haja só meu corpo nu\nJunto

In [ ]:
def getSongData(url):
  soup = BeautifulSoup(urlopen(url), 'html.parser')

  windowPageArgs = soup.find("script",{"id":"head_scripts_vars"}).text.split('__pageArgs=')[1].split(";")[0] 

  views = -1
  views_div = soup.find("div","cnt-info_exib")
  if views_div is not None:
    views = int(views_div.find("b").getText().replace('.',''))

  stanzas = []
  for p in soup.find("div","cnt-letra p402_premium").findAll("p"):
    stanzas.append(
        str(p)
          .replace("<p>","")
          .replace("</p>","")
          .replace("<br>","\n")
          .replace("<br/>","\n")
          .replace("</br>","\n")
      )

  lyrics = "\n\n".join(stanzas)

  return {"windowPageArgs":windowPageArgs,"views":views,"lyrics":lyrics}


In [ ]:
# Todo o dataset:

In [ ]:
url = f'https://www.letras.com/gilberto-gil/'
page_info=urlopen(url)
soup = BeautifulSoup(page_info, 'html.parser')
data = soup.find('div', "cnt-list--alp")
songs = data.findAll('li')

In [ ]:
songs_list = []
for li in songs:
  songs_list.append({
      'name': li['data-name'],
      'letras_id':li['data-id'],
      'url':li['data-shareurl'],
      'text':li['data-sharetext'],
  })

In [ ]:
songsDF = pd.DataFrame(songs_list)

In [ ]:
songsDF.loc[songsDF['name'].duplicated(keep='last')]

,name,letras_id,url,text
480,Punk da Periferia,16135,https://www.letras.com/gilberto-gil/16135/,Punk da Periferia de Gilberto Gil
515,Retiros Espirituais,2025824,https://www.letras.com/gilberto-gil/retidos-es...,Retiros Espirituais de Gilberto Gil
607,Vamos Fugir,16137,https://www.letras.com/gilberto-gil/16137/,Vamos Fugir de Gilberto Gil


In [ ]:
songsDF['individualScrape'] = songsDF.apply(lambda x: getSongData(x['url']), axis = 1)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [ ]:
songsDF

,name,letras_id,url,text,individualScrape
0,17 Léguas e Meia,1571967,https://www.letras.com/gilberto-gil/1571967/,17 Léguas e Meia de Gilberto Gil,"{'windowPageArgs': '{""album_name"":""Gilberto Gi..."
1,17 Na Corrente,2167336,https://www.letras.com/gilberto-gil/17-na-corr...,17 Na Corrente de Gilberto Gil,"{'windowPageArgs': '{""album_name"":""O Sol de Os..."
2,2001,1179389,https://www.letras.com/gilberto-gil/1179389/,2001 de Gilberto Gil,"{'windowPageArgs': '{""album_name"":""Gilberto Gi..."
3,- Aqui Agora,3160604,https://www.letras.com/gilberto-gil/aqui-agora/,- Aqui Agora de Gilberto Gil,"{'windowPageArgs': '{""artist_name"":""Gilberto G..."
4,A Abóbada da Vida,316672,https://www.letras.com/gilberto-gil/316672/,A Abóbada da Vida de Gilberto Gil,"{'windowPageArgs': '{""artist_name"":""Gilberto G..."
...,...,...,...,...,...
633,Yá Olokum,2025819,https://www.letras.com/gilberto-gil/ya-olokum/,Yá Olokum de Gilberto Gil,"{'windowPageArgs': '{""album_name"":""Parabolicam..."
634,Yo Vengo a Oferecer Mi Corazon,1571966,https://www.letras.com/gilberto-gil/1571966/,Yo Vengo a Oferecer Mi Corazon de Gilberto Gil,"{'windowPageArgs': '{""album_name"":""Gil e Milto..."
635,Zabelê,586713,https://www.letras.com/gilberto-gil/586713/,Zabelê de Gilberto Gil,"{'windowPageArgs': '{""artist_name"":""Gilberto G..."
636,Zoilógico,586714,https://www.letras.com/gilberto-gil/586714/,Zoilógico de Gilberto Gil,"{'windowPageArgs': '{""artist_name"":""Gilberto G..."


In [ ]:
# songsDF['individualScrape'].apply(lambda x: x['windowPageArgs'])
songsDF['windowPageArgs'] = songsDF['individualScrape'].apply(lambda x: x['windowPageArgs'])
songsDF['views'] = songsDF['individualScrape'].apply(lambda x: x['views'])
songsDF['lyrics'] = songsDF['individualScrape'].apply(lambda x: x['lyrics'])
# songsDF['composicao'] = songsDF['individualScrape'].apply(lambda x: x['composicao'])

In [ ]:
songsDF.sort_values(by='views', ascending = False).head(5)

,name,letras_id,url,text,individualScrape,windowPageArgs,views,lyrics
57,Aquele Abraço,16138,https://www.letras.com/gilberto-gil/16138/,Aquele Abraço de Gilberto Gil,"{'windowPageArgs': '{""album_name"":""Gilberto Gi...","{""album_name"":""Gilberto Gil"",""artist_name"":""Gi...",35941,O Rio de Janeiro continua lindo\nO Rio de Jane...
52,Andar Com Fé,46184,https://www.letras.com/gilberto-gil/46184/,Andar Com Fé de Gilberto Gil,"{'windowPageArgs': '{""album_name"":""Um Banda um...","{""album_name"":""Um Banda um"",""artist_name"":""Gil...",28743,Andá com fé eu vou\nQue a fé não costuma faiá\...
180,Drão,16133,https://www.letras.com/gilberto-gil/16133/,Drão de Gilberto Gil,"{'windowPageArgs': '{""album_name"":""Um Banda um...","{""album_name"":""Um Banda um"",""artist_name"":""Gil...",21734,Drão!\nO amor da gente é como um grão\nUma sem...
583,Toda Menina Baiana,46249,https://www.letras.com/gilberto-gil/46249/,Toda Menina Baiana de Gilberto Gil,"{'windowPageArgs': '{""album_name"":""Realce"",""ar...","{""album_name"":""Realce"",""artist_name"":""Gilberto...",17566,Toda menina baiana tem\nUm santo que Deus dá\n...
205,Esperando Na Janela,46204,https://www.letras.com/gilberto-gil/46204/,Esperando Na Janela de Gilberto Gil,"{'windowPageArgs': '{""album_name"":""Warner 25 A...","{""album_name"":""Warner 25 Anos"",""artist_name"":""...",16645,Ainda me lembro do seu caminhar\nSeu jeito de ...


In [ ]:
songsDF.loc[songsDF.views == -1]

,name,letras_id,url,text,individualScrape,windowPageArgs,views,lyrics
321,Mané João,3421587,https://www.letras.com/gilberto-gil/mane-joao/,Mané João de Gilberto Gil,"{'windowPageArgs': '{""artist_name"":""Gilberto G...","{""artist_name"":""Gilberto Gil"",""dns"":""gilberto-...",-1,Lá na gafieira\nDe Mané João\nToda brincadeira...


In [ ]:
songsDF.to_csv('df_coleta_letras_gilberto_gil.csv', index=False)

## 2. Criação de nuvem de palavras <a name="Nuvem"></a>

In [ ]:
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import PorterStemmer

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# lista de stopwords em pt-br

stopwords_ptbr = stopwords.words('portuguese')
stopwords_eng = stopwords.words('english')
stopwords_fra = stopwords.words('french')

# ampliar a lista de stopwords com outras expressões que estão presentes nas letras das músicas
novas_palavras = ['oh', 'sei', 'cada', 'tá', 'dum', 'ó', 'tê', 'obá', 'sobre', 'quém', 'bat', 'dizer', 'ter', 'ficar', 'fazer' 'gente', 'grande', 'qualquer', 'dentro', 'alguém', 'diz', 'disse', 'vê', 'aí', 'outra', 'então', 'lado', 'tanta', 'iê', 'quase', 'pois']

stopwords_ptbr.extend(novas_palavras)

In [ ]:
# criar função processingTxt para preservar termos compostos e separar palavras que estão unidas e não deveriam estar

def processingTxt(txt):
    return txt.replace('expresso 2222', 'expresso_2222').replace('nova deli', 'nova_deli').replace('a gente', 'a_gente').replace('favelainferno', 'favela inferno').replace('pensamentoé', 'pensamento é')

In [ ]:
songsDF['name'].count()

638

In [ ]:
# excluir músicas duplicadas
songsDF_unicas = songsDF.drop_duplicates(subset='name')
songsDF_unicas.head(2)

,name,letras_id,url,text,individualScrape,windowPageArgs,views,lyrics
0,17 Léguas e Meia,1571967,https://www.letras.com/gilberto-gil/1571967/,17 Léguas e Meia de Gilberto Gil,"{'windowPageArgs': '{""album_name"":""Gilberto Gi...","{""album_name"":""Gilberto Gil"",""artist_name"":""Gi...",126,Eu já andei sem parar dezessete légua e meia\n...
1,17 Na Corrente,2167336,https://www.letras.com/gilberto-gil/17-na-corr...,17 Na Corrente de Gilberto Gil,"{'windowPageArgs': '{""album_name"":""O Sol de Os...","{""album_name"":""O Sol de Oslo"",""artist_name"":""G...",43,Macaco não é valente\nDança aí 17 na corrente....


In [ ]:
# deixar letras de músicas em caixa baixa, criar a coluna '[letras_musicas] e aplicar função processingTxt

songsDF_unicas['letras_musicas'] = songsDF_unicas['lyrics'].str.lower()
songsDF_unicas['letras_musicas'] = songsDF_unicas['letras_musicas'].fillna('-').apply(lambda x: processingTxt(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
def encontrarPalavras(letras):

    # criar lista com letras das músicas após retirar o separador
    letras_separadas = list()

    for letra_musica in letras:
      letra_musica = letra_musica.lower()
      letra_musica_separada = letra_musica.split('\n')
      letras_separadas.append(letra_musica_separada)

    # criar variável para tokenizador
    tokenizador = nltk.WhitespaceTokenizer()

    # criar lista com letras das músicas após tokenização
    tokens = list()

    for letra in letras_separadas:
      for frase in letra:
        frase_tokenizada = tokenizador.tokenize(frase)
        tokens.append(frase_tokenizada)

    # criar lista com palavras sem pontuação e caracteres especiais
    token_sem_pontuacao = list()

    for token in tokens:
      for palavra in token:
        novo_token = palavra.translate ({ord(c): "" for c in "'…‘“”!@#$%^&*()[]{};:.,/–-<>?\|`~=+"})
        token_sem_pontuacao.append(novo_token)

    # criar lista com palavras fora da lista de stopwords
    palavras_fora_stopwords = list()

    for palavra in token_sem_pontuacao:
      if palavra not in stopwords_ptbr and palavra not in stopwords_eng and palavra not in stopwords_fra:
        palavras_fora_stopwords.append(palavra)

    # criar dataframe com as palavras que não estão na lista de stopwords
    df_palavras_coluna = pd.DataFrame(palavras_fora_stopwords)

    df_palavras_coluna.rename(columns={0: 'palavras'}, inplace=True)

    # criação da coluna 'repetições'
    df_palavras_coluna['repetições'] = 1
        
    # remover underline
    df_palavras_coluna['palavras'] = df_palavras_coluna['palavras'].str.replace('_', ' ')
    df_palavras_coluna['palavras'] = df_palavras_coluna['palavras'].str.replace('"', '')

    return df_palavras_coluna

In [ ]:
nuvem_de_palavras = encontrarPalavras(songsDF_unicas['letras_musicas'])

In [ ]:
df_nuvem = nuvem_de_palavras.groupby('palavras').sum().sort_values(by='repetições', ascending=False).reset_index()
df_nuvem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10303 entries, 0 to 10302
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   palavras    10303 non-null  object
 1   repetições  10303 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 161.1+ KB


In [ ]:
df_nuvem.to_csv('df_nuvem_de_palavras.csv', index=False)

In [ ]:
df_nuvem['repetições'].unique()

array([647, 411, 248, 247, 235, 227, 225, 224, 210, 195, 192, 188, 187,
       184, 180, 166, 161, 160, 158, 156, 153, 152, 150, 148, 140, 138,
       134, 132, 131, 121, 116, 115, 113, 111, 110, 109, 108, 106, 101,
        99,  98,  91,  90,  89,  88,  87,  86,  83,  82,  79,  77,  73,
        72,  71,  70,  68,  66,  65,  64,  63,  62,  61,  60,  59,  56,
        55,  54,  53,  52,  51,  50,  49,  48,  47,  46,  45,  44,  43,
        42,  41,  40,  39,  38,  37,  36,  35,  34,  33,  32,  31,  30,
        29,  28,  27,  26,  25,  24,  23,  22,  21,  20,  19,  18,  17,
        16,  15,  14,  13,  12,  11,  10,   9,   8,   7,   6,   5,   4,
         3,   2,   1])

In [ ]:
palavras_40_repeticoes = df_nuvem[df_nuvem['repetições'].isin([411, 247, 224, 195, 188, 158, 156, 153, 152, 140, 138, 134, 132, 121, 113, 108, 107, 101,  91,  90,  88,  87,  86,  83,  82,  79, 77,  73,  72,  71,  68,  66,  65,  64,  63,  61,  60,  58,  57, 56,  55,  54,  53,  52,  51,  50,  49,  48,  47,  46,  45,  44, 43,  42,  41,  40])]

In [ ]:
palavras_40_repeticoes.to_csv('df_nuvem_palavras_40_repeticoes.csv', index=False)

In [ ]:
# informações palavra mais repetida
musicas_amor = songsDF_unicas[songsDF_unicas['letras_musicas'].str.contains('amor')]

In [ ]:
print('A palavra mais frequente, "Amor", apareceu em', musicas_amor['name'].nunique(), 'do total de', songsDF_unicas['name'].nunique(), 'músicas coletadas.')

A palavra mais frequente, "Amor", apareceu em 220 do total de 635 músicas coletadas.


In [ ]:
print(df_nuvem['palavras'].nunique(), 'ocorrências (palavras, formas ou vocábulos)')

10303 ocorrências (palavras, formas ou vocábulos)


In [ ]:
print(df_nuvem[df_nuvem['repetições'] != 1]['palavras'].nunique(), 'palavras se repetem')

4857 palavras se repetem


In [ ]:
print(df_nuvem[df_nuvem['repetições'] == 1]['palavras'].nunique(), 'palavras têm uma única ocorrência.')

5446 palavras têm uma única ocorrência.
